# PyNeoInstance Ingestion Demo

Data sourced from: https://www.kaggle.com/datasets/mexwell/countries-states-and-cities-around-the-world

In [1]:
import pandas as pd
from pyneoinstance import Neo4jInstance, load_yaml_file

/var/folders/7_/vqs74z3j5hscgzbt0ydbmqyw0000gq/T/ipykernel_80155/1670258370.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Prepare Data

### Assign File Locations

In [2]:
cities = "data/cities.csv"
countries = "data/countries.csv"
states = "data/states.csv"

### Load Into DataFrames

In [3]:
cities_df = pd.read_csv(cities)
countries_df = pd.read_csv(countries)
states_df = pd.read_csv(states)

## Load Configuration

In [4]:
config = load_yaml_file("config.yaml")
db_info = config['db_info']
constraints = config['initializing_queries']['constraints']
indexes = config['initializing_queries']['indexes']
node_load_queries = config['loading_queries']['nodes']
relationship_load_queries = config['loading_queries']['relationships']

## Load Data Into Graph

### Connect to Graph

In [5]:
graph = Neo4jInstance(db_info['uri'], db_info['user'], db_info['password'])

### Create Constraints & Indexes

In [6]:
try:
    graph.execute_write_queries(database=db_info['database'], queries=list(constraints.values()))
except Exception as e:
    print(e)

try:
    graph.execute_write_queries(database=db_info['database'], queries=list(indexes.values()))
except Exception as e:
    print(e)

'NoneType' object has no attribute 'values'


### Load Nodes

In [7]:
def get_partition(data: pd.DataFrame, batch_size: int = 500) -> int:
    """
    Determine the data partition based on the desired batch size.
    """
    
    partition = int(len(data) / batch_size)
    print("partition: "+str(partition if partition > 1 else 1))
    return partition if partition > 1 else 1

In [8]:
for data, query in list(zip([cities_df, countries_df, states_df], ['cities', 'countries', 'states'])):
    res = graph.execute_write_query_with_data(database=db_info['database'], 
                                                data=data, 
                                                query=node_load_queries[query], 
                                                partitions=get_partition(data, batch_size=500),
                                                parallel=True,
                                                workers=2)
    print(res)

partition: 300


/Users/alexandergilmore/Documents/projects/my-tutorials/pyneoinstance/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'labels_added': 150454, 'nodes_created': 150454, 'properties_set': 300908}
partition: 1


/Users/alexandergilmore/Documents/projects/my-tutorials/pyneoinstance/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'labels_added': 436, 'nodes_created': 436, 'properties_set': 936}
partition: 10


/Users/alexandergilmore/Documents/projects/my-tutorials/pyneoinstance/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'labels_added': 5077, 'nodes_created': 5077, 'properties_set': 10154}


### Load Relationships

In [9]:
for data, query in list(zip([cities_df, states_df], ['cities', 'states'])):
    res = graph.execute_write_query_with_data(  database=db_info['database'], 
                                                data=data, 
                                                query=relationship_load_queries[query], 
                                                partitions=get_partition(data, batch_size=500),
                                                parallel=True,
                                                workers=2)
    print(res)

res = graph.execute_write_query_with_data(  database=db_info['database'], 
                                                data=countries_df, 
                                                query=relationship_load_queries["countries"], 
                                                partitions=get_partition(data, batch_size=500))
print(res)

partition: 300


/Users/alexandergilmore/Documents/projects/my-tutorials/pyneoinstance/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'relationships_created': 150454}
partition: 10


/Users/alexandergilmore/Documents/projects/my-tutorials/pyneoinstance/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'relationships_created': 5077}
partition: 10


/Users/alexandergilmore/Documents/projects/my-tutorials/pyneoinstance/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


{'relationships_created': 524}
